In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ctrace
from tqdm import tqdm
from ctrace.simulation import *
from ctrace.utils import *

from ctrace.dataset import load_graph, load_sir_path
import concurrent.futures

In [41]:
with open("with_optimal.csv", "r") as csv:
    df = pd.read_csv(csv)
df.head()

,Unnamed: 0,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,...,num_cross_edges,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,optimal_MIP_value,mip_ratio
0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,90.850904,25,28,...,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407,0,730.259302,1.030885
1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,90.716560,20,28,...,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407,0,730.259302,1.030701
2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,69.442667,256,28,...,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407,0,730.259302,1.001569
3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,69.491244,252,28,...,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407,0,730.259302,1.001636
4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,71.084567,-1,28,...,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407,0,730.259302,1.003817


In [42]:
list(df)

['Unnamed: 0',
 'G',
 'p',
 'budget',
 'from_cache',
 'method',
 'trial_id_x',
 'mip_value',
 'greedy_intersection',
 'maxD',
 'I_size',
 'v1_size',
 'v2_size',
 'num_cross_edges',
 'duration',
 'updated_MIP_x',
 'v1_sum_values',
 'min_exposed_mean',
 'min_exposed_stddev',
 'updated_min_exposed_value',
 'trial_id_y',
 'optimal_MIP_value',
 'mip_ratio']

In [43]:
cols = [
 'budget',
 'from_cache',
 'method',
 'trial_id_x','maxD','I_size',
 'v1_size',
 'v2_size',
 'num_cross_edges','duration','updated_min_exposed_value', 'mip_ratio']

In [44]:
df = df[cols]

In [45]:
df = df.sort_values(by="v1_size")

In [46]:
df.head()

,budget,from_cache,method,trial_id_x,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_min_exposed_value,mip_ratio
1524,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,23,529,2808,6472,8273,0.025445,306.334672,1.003850
5526,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,23,529,2808,6472,8273,0.016595,299.134672,1.003385
5525,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,23,529,2808,6472,8273,0.388629,303.534672,1.000000
5524,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,23,529,2808,6472,8273,0.016042,299.934672,1.003385
5523,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,23,529,2808,6472,8273,0.389772,299.534672,1.000392


In [ ]:
df = df[df["budget"] >= ]

In [47]:
import numpy as np
chunks = np.array_split(df, 4)

In [48]:
chunks[1].head()

,budget,from_cache,method,trial_id_x,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_min_exposed_value,mip_ratio
6727,700,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,60,787,11279,39279,118975,19.001118,2074.002821,1.000000
10727,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,60,787,11279,39279,118975,20.665809,2017.802821,1.000000
8725,800,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,60,787,11279,39279,118975,10.561098,2070.002821,1.013699
10726,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,60,787,11279,39279,118975,19.256921,2014.602821,1.000000
10724,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,60,787,11279,39279,118975,14.083605,2066.802821,1.015809


In [38]:
len(chunks[0])

3500

In [39]:
chunks[0]

,budget,from_cache,method,trial_id_x,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_min_exposed_value,mip_ratio
1524,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,23,529,2808,6472,8273,0.025445,306.334672,1.003850
5526,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,23,529,2808,6472,8273,0.016595,299.134672,1.003385
5525,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,23,529,2808,6472,8273,0.388629,303.534672,1.000000
5524,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,23,529,2808,6472,8273,0.016042,299.934672,1.003385
5523,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,23,529,2808,6472,8273,0.389772,299.534672,1.000392
...,...,...,...,...,...,...,...,...,...,...,...,...
12725,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,60,787,11279,39279,118975,12.758492,2082.002821,1.016771
4723,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,60,787,11279,39279,118975,10.715024,2127.402821,1.013087
12724,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,60,787,11279,39279,118975,12.331257,2070.602821,1.018112
4721,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,60,787,11279,39279,118975,5.479658,2284.802821,1.068347


In [14]:
# MILP optimal gap 
c0 = chunks[0]

In [32]:
for i, chunk in enumerate(chunks):
    print(chunk[chunk["method"] == "dependent"].mean())

budget                         699.428571
trial_id_x                       0.499429
maxD                            36.352000
I_size                        1169.240000
v1_size                       7125.192000
v2_size                      17005.352000
num_cross_edges              36679.720000
duration                         3.453459
updated_min_exposed_value     1022.112935
mip_ratio                        1.004406
dtype: float64
budget                         700.571429
trial_id_x                       0.500571
maxD                            44.160000
I_size                        2330.344000
v1_size                      15416.856000
v2_size                      27464.584000
num_cross_edges              85411.640000
duration                        10.791684
updated_min_exposed_value     2703.433527
mip_ratio                        1.006869
dtype: float64
budget                         699.771689
trial_id_x                       0.500000
maxD                            36.680365
I_si

In [35]:
methods = ["dependent", "greedy_weighted", "random", "mip_gurobi"]

In [36]:
for i, chunk in enumerate(chunks):
    for method in methods:
        values = chunk[chunk["method"] == method].mean()
        for var in ["mip_ratio", "updated_min_exposed_value", "duration"]:
            print(f"{i}, {method}, {var}, {values[var]}")


0, dependent, mip_ratio, 1.0044064965361603
0, dependent, updated_min_exposed_value, 1022.1129346377736
0, dependent, duration, 3.453458837200037
0, greedy_weighted, mip_ratio, 1.0058463863741658
0, greedy_weighted, updated_min_exposed_value, 1006.787448923488
0, greedy_weighted, duration, 0.159286642045768
0, random, mip_ratio, 1.0523970401408496
0, random, updated_min_exposed_value, 1085.3351096494528
0, random, duration, 1.8762918471992425
0, mip_gurobi, mip_ratio, 1.0
0, mip_gurobi, updated_min_exposed_value, 1011.2199648439041
0, mip_gurobi, duration, 6.404478692465601
1, dependent, mip_ratio, 1.0068692436855513
1, dependent, updated_min_exposed_value, 2703.4335265075
1, dependent, duration, 10.791684364535971
1, greedy_weighted, mip_ratio, 1.0054088642007801
1, greedy_weighted, updated_min_exposed_value, 2669.1055265075
1, greedy_weighted, duration, 0.3728836516937403
1, random, mip_ratio, 1.036784897402164
1, random, updated_min_exposed_value, 2814.0317972819894
1, random, durat